**Experiment 1**

This notebook contains experiments conducted in order to find out whether having a smaller but more focused dataset improves the word2vec model's capability of extracting terms related to the ASDPTO and HPO ontologies.
The focused autism dataset is compared with a dataset expanded with a non-related disease(asthma) and with a related disease(dyslexia)

In [8]:
%load_ext autoreload
%autoreload 2

import word2vec
import gensim
import sys
sys.path.append("../../")
import pm_parser
import evaluation

def train_model(dataset="Training", model='vectors-phrase.bin', th1=100, th2=50,cbow=0, size=200, min_count=1, window=10, sample="1e-5", negative=25, threads=20, iters=15):
    !sed -e "s/’/'/g" -e "s/′/'/g" -e "s/''/ /g" < {dataset} | tr -c "A-Za-z'_ \n" " " > Training-norm0
    print('Finished preprocessing 1')
    word2vec.word2phrase('Training-norm0', 'Training-norm0-phrase0', threshold=th1, verbose=True)
    print('Finished word2phrase 1')
    word2vec.word2phrase('Training-norm0-phrase0', 'Training-norm0-phrase1', threshold=th2, verbose=True)
    print('Finished word2phrase 2')
    !tr A-Z a-z < Training-norm0-phrase1 > Training-norm1-phrase1
    print('Finished preprocessing 2')
    word2vec.word2vec('Training-norm1-phrase1', model, cbow=cbow, size=size, window=window, min_count=min_count, sample=sample, negative=negative, threads=20, iter_=iters, verbose=True)
    print('Finished model training')
    !rm Training-norm0 Training-norm0-phrase0 Training-norm0-phrase1 Training-norm1-phrase1
    print('Removed reduntant files')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
def train_phrase(dataset="Training",out="Training-norm1-phrase1", th1=100, th2=50):
    !sed -e "s/’/'/g" -e "s/′/'/g" -e "s/''/ /g" < {dataset} | tr -c "A-Za-z'_ \n" " " > Training-norm0
    print('Finished preprocessing 1')
    word2vec.word2phrase('Training-norm0', 'Training-norm0-phrase0', threshold=th1, verbose=True)
    print('Finished word2phrase 1')
    word2vec.word2phrase('Training-norm0-phrase0', 'Training-norm0-phrase1', threshold=th2, verbose=True)
    print('Finished word2phrase 2')
    !tr A-Z a-z < Training-norm0-phrase1 > {out}
    print('Finished preprocessing 2')
    !rm Training-norm0 Training-norm0-phrase0 Training-norm0-phrase1
    print('Removed reduntant files')

*1a) Autism dataset*

In this experiment the word2vec model is trained on abstracts annotated with **only** autism-related mesh terms.


In [2]:
dataset = '/afs/inf.ed.ac.uk/group/project/biomednlp/msc2019/Shibo/word2vec/examples/Training'

In [3]:
train_model('../../../Shibo/word2vec/examples/Training', 'autism_model.bin')

Finished preprocessing 1
Starting training using file Training-norm0
Words processed: 4400K     Vocab size: 646K  
Vocab size (unigrams + bigrams): 402225
Words in train file: 4475947
Finished word2phrase 1
Starting training using file Training-norm0-phrase0
Words processed: 4000K     Vocab size: 728K  
Vocab size (unigrams + bigrams): 444758
Words in train file: 4091230
Finished word2phrase 2
Finished preprocessing 2
Starting training using file Training-norm1-phrase1
Vocab size: 57901
Words in train file: 3843255
Alpha: 0.000002  Progress: 100.04%  Words/thread/sec: 15.62k  Finished model training
Removed reduntant files


In [ ]:
#evaluate here

*1b) Autism + Dyslexia dataset*

In this experiment the word2vec model is trained on abstracts annotated with two related diseases' (autism or dyslexia) mesh terms.

In [4]:
train_model('../../../Shibo/dataset/dysiexia_Training.txt', 'dyslexia_model.bin')

Finished preprocessing 1
Starting training using file Training-norm0
Words processed: 1500K     Vocab size: 251K  
Vocab size (unigrams + bigrams): 155301
Words in train file: 1528798
Finished word2phrase 1
Starting training using file Training-norm0-phrase0
Words processed: 1400K     Vocab size: 273K  
Vocab size (unigrams + bigrams): 166372
Words in train file: 1425128
Finished word2phrase 2
Finished preprocessing 2
Starting training using file Training-norm1-phrase1
Vocab size: 23749
Words in train file: 1346446
Alpha: 0.000002  Progress: 100.07%  Words/thread/sec: 18.40k  Finished model training
Removed reduntant files


In [ ]:
#evaluate here

*1c) Autism + Asthma dataset*

In this experiment the word2vec model is trained on abstracts annotated with two weakly related diseases' (autism or asthma) mesh terms.

In [5]:
train_model('../../../Shibo/dataset/Asthma_Training.txt', 'asthma_model.bin')

Finished preprocessing 1
Starting training using file Training-norm0
Words processed: 1400K     Vocab size: 256K  
Vocab size (unigrams + bigrams): 156248
Words in train file: 1430491
Finished word2phrase 1
Starting training using file Training-norm0-phrase0
Words processed: 1300K     Vocab size: 278K  
Vocab size (unigrams + bigrams): 167174
Words in train file: 1320260
Finished word2phrase 2
Finished preprocessing 2
Starting training using file Training-norm1-phrase1
Vocab size: 25472
Words in train file: 1254264
Alpha: 0.000002  Progress: 100.09%  Words/thread/sec: 17.66k  Finished model training
Removed reduntant files


In [ ]:
#evaluate here

*1d) Mixed dataset*

In this experiment the word2vec model is trained on abstracts annotated with a mixed dataset' (autism + asthma + dyslexia) mesh terms.

In [4]:
word2vec.word2vec('../../../Shibo/dataset/Training-norm1-phrase1', 'mixed_model.bin', cbow=0, size=150, window=10, min_count=1, sample="1e-5", negative=25, threads=20, iter_=15, verbose=True)
print('Finished model training')

Starting training using file ../../../Shibo/dataset/Training-norm1-phrase1
Vocab size: 57583
Words in train file: 3850548
Alpha: 0.000002  Progress: 100.05%  Words/thread/sec: 16.54k  Finished model training


In [ ]:
#evaluate here

**FULL FOLDER EVALUATION**

In [6]:
evaluation.evaluate_folder('/', 'experiment1_gold_results.csv', ext='.bin',annotated_testset_file="../../annotated_GOLD_testset.json", phenotypes_pickle="../../onto_tokens.pickle", mwe_tokens_pickle="../../mwe_tokens.pickle")

  0%|          | 0/3 [00:00<?, ?it/s]

Number of models to evaluate: 3


/afs/inf.ed.ac.uk/user/s12/s1248922/miniconda3/envs/mlp/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL

100%|██████████| 3/3 [05:38<00:00, 102.49s/it]]

****finished****


In [7]:
evaluation.evaluate_folder('/', 'experiment1_ncbo_results.csv', ext='.bin',annotated_testset_file="../../annotated_testset.json", phenotypes_pickle="../../onto_tokens.pickle", mwe_tokens_pickle="../../mwe_tokens.pickle")

  0%|          | 0/3 [00:00<?, ?it/s]

Number of models to evaluate: 3



 43%|████▎     | 366/853 [01:15<01:50,  4.42it/s]


 87%|████████▋ | 741/853 [02:36<00:36,  3.11it/s]


 27%|██▋       | 232/853 [01:22<03:47,  2.73it/s]


...

  7%|▋         | 56/853 [00:10<02:35,  5.12it/s]


 53%|█████▎    | 452/853 [01:18<01:12,  5.56it/s]


 99%|█████████▉| 848/853 [02:30<00:00,  5.84it/s]


100%|██████████| 3/3 [10:16<00:00, 193.07s/it]/s]

****finished****


In [6]:
evaluation.evaluate_folder('/', 'experiment1_gold_results.csv', ext='mixed_model.bin',annotated_testset_file="../../annotated_GOLD_testset.json", phenotypes_pickle="../../onto_tokens.pickle", mwe_tokens_pickle="../../mwe_tokens.pickle")

  0%|          | 0/1 [00:00<?, ?it/s]

Number of models to evaluate: 1


/afs/inf.ed.ac.uk/user/s12/s1248922/miniconda3/envs/mlp/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL

100%|██████████| 1/1 [02:43<00:00, 163.85s/it]]

****finished****


In [9]:
evaluation.evaluate_folder('/', 'experiment1_ncbo_results.csv', ext='mixed_model.bin',annotated_testset_file="../annotated_mixed_testset.json", phenotypes_pickle="../../onto_tokens.pickle", mwe_tokens_pickle="../../mwe_tokens.pickle")

  0%|          | 0/1 [00:00<?, ?it/s]

Number of models to evaluate: 1



 13%|█▎        | 337/2551 [01:32<04:51,  7.59it/s]


 26%|██▋       | 673/2551 [03:20<14:43,  2.13it/s]


 39%|███▉      | 995/2551 [05:47<09:12,  2.82it/s]


...

 65%|██████▌   | 1659/2551 [11:52<10:39,  1.39it/s]


 78%|███████▊  | 2002/2551 [14:14<03:11,  2.87it/s]


 92%|█████████▏| 2355/2551 [16:28<01:46,  1.85it/s]


100%|██████████| 1/1 [17:40<00:00, 1060.92s/it]t/s]

****finished****


**GOLD**

In [9]:
train_model('../../../Shibo/word2vec/examples/Training', 'autism_model_gold.bin', window=20, min_count=0, size=150)

Finished preprocessing 1
Starting training using file Training-norm0
Words processed: 4400K     Vocab size: 646K  
Vocab size (unigrams + bigrams): 402225
Words in train file: 4475947
Finished word2phrase 1
Starting training using file Training-norm0-phrase0
Words processed: 4000K     Vocab size: 728K  
Vocab size (unigrams + bigrams): 444758
Words in train file: 4091230
Finished word2phrase 2
Finished preprocessing 2
Starting training using file Training-norm1-phrase1
Vocab size: 57901
Words in train file: 3843255
Alpha: 0.000002  Progress: 100.04%  Words/thread/sec: 11.23k  Finished model training
Removed reduntant files


In [10]:
train_model('../../../Shibo/dataset/dysiexia_Training.txt', 'dyslexia_model_gold.bin', window=20, min_count=0, size=150)

Finished preprocessing 1
Starting training using file Training-norm0
Words processed: 1500K     Vocab size: 251K  
Vocab size (unigrams + bigrams): 155301
Words in train file: 1528798
Finished word2phrase 1
Starting training using file Training-norm0-phrase0
Words processed: 1400K     Vocab size: 273K  
Vocab size (unigrams + bigrams): 166372
Words in train file: 1425128
Finished word2phrase 2
Finished preprocessing 2
Starting training using file Training-norm1-phrase1
Vocab size: 23749
Words in train file: 1346446
Alpha: 0.000005  Progress: 100.03%  Words/thread/sec: 13.56k  Finished model training
Removed reduntant files


In [12]:
train_model('../../../Shibo/dataset/Asthma_Training.txt', 'asthma_model_gold.bin', window=20, min_count=0, size=150)

Finished preprocessing 1
Starting training using file Training-norm0
Words processed: 1400K     Vocab size: 256K  
Vocab size (unigrams + bigrams): 156248
Words in train file: 1430491
Finished word2phrase 1
Starting training using file Training-norm0-phrase0
Words processed: 1300K     Vocab size: 278K  
Vocab size (unigrams + bigrams): 167174
Words in train file: 1320260
Finished word2phrase 2
Finished preprocessing 2
Starting training using file Training-norm1-phrase1
Vocab size: 25472
Words in train file: 1254264
Alpha: 0.000002  Progress: 100.09%  Words/thread/sec: 13.30k  Finished model training
Removed reduntant files


In [14]:
word2vec.word2vec('../../../Shibo/dataset/Training-norm1-phrase1', 'mixed_model_gold.bin', cbow=0, size=150, window=200, min_count=0, sample="1e-5", negative=25, threads=20, iter_=15, verbose=True)
print('Finished model training')

Starting training using file ../../../Shibo/dataset/Training-norm1-phrase1
Vocab size: 57583
Words in train file: 3850548
Alpha: 0.000002  Progress: 100.05%  Words/thread/sec: 4.06k  Finished model training


In [13]:
evaluation.evaluate_folder('/', 'experiment1_gold_results_gfix.csv', ext='gold.bin',annotated_testset_file="../../annotated_GOLD_testset.json", phenotypes_pickle="../../onto_tokens.pickle", mwe_tokens_pickle="../../mwe_tokens.pickle")

  0%|          | 0/3 [00:00<?, ?it/s]

Number of models to evaluate: 3


/afs/inf.ed.ac.uk/user/s12/s1248922/miniconda3/envs/mlp/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL

100%|██████████| 3/3 [01:52<00:00, 42.66s/it]s]

****finished****


In [15]:
evaluation.evaluate_folder('/', 'experiment1_gold_results_gfix.csv', ext='mixed_model_gold.bin',annotated_testset_file="../../annotated_GOLD_testset.json", phenotypes_pickle="../../onto_tokens.pickle", mwe_tokens_pickle="../../mwe_tokens.pickle")

  0%|          | 0/1 [00:00<?, ?it/s]

Number of models to evaluate: 1


/afs/inf.ed.ac.uk/user/s12/s1248922/miniconda3/envs/mlp/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL

100%|██████████| 1/1 [01:01<00:00, 61.32s/it]t]

****finished****


**PHRASES FOR GLOVE**

In [20]:
train_phrase('../../../Shibo/word2vec/examples/Training', out='autism-phrase1')

Finished preprocessing 1
Starting training using file Training-norm0
Words processed: 4400K     Vocab size: 646K  
Vocab size (unigrams + bigrams): 402225
Words in train file: 4475947
Finished word2phrase 1
Starting training using file Training-norm0-phrase0
Words processed: 4000K     Vocab size: 728K  
Vocab size (unigrams + bigrams): 444758
Words in train file: 4091230
Finished word2phrase 2
Finished preprocessing 2
Removed reduntant files


In [21]:
train_phrase('../../../Shibo/dataset/dysiexia_Training.txt', out='dyslexia-phrase1')

Finished preprocessing 1
Starting training using file Training-norm0
Words processed: 1500K     Vocab size: 251K  
Vocab size (unigrams + bigrams): 155301
Words in train file: 1528798
Finished word2phrase 1
Starting training using file Training-norm0-phrase0
Words processed: 1400K     Vocab size: 273K  
Vocab size (unigrams + bigrams): 166372
Words in train file: 1425128
Finished word2phrase 2
Finished preprocessing 2
Removed reduntant files


In [22]:
train_phrase('../../../Shibo/dataset/Asthma_Training.txt', out='asthma-phrase1')

Finished preprocessing 1
Starting training using file Training-norm0
Words processed: 1400K     Vocab size: 256K  
Vocab size (unigrams + bigrams): 156248
Words in train file: 1430491
Finished word2phrase 1
Starting training using file Training-norm0-phrase0
Words processed: 1300K     Vocab size: 278K  
Vocab size (unigrams + bigrams): 167174
Words in train file: 1320260
Finished word2phrase 2
Finished preprocessing 2
Removed reduntant files
